## Retrieve interactions from BioGrid

In [87]:
import numpy as np
import pandas as pd
import os
import sys
import datetime
import time
import pickle
# import idconversion as idc
# from idconversion import idconversion as idc
from urllib.error import HTTPError

## Helper Functions


In [3]:
def time_stamper(tdate=datetime.datetime.now()):
    fmtdate = tdate.strftime('%Y-%m-%d.%H:%M:%S')
    return fmtdate

In [4]:
time_stamper(datetime.datetime.now())

'2019-06-27.14:57:27'

In [5]:
# dir(idc)


In [7]:
!ls 

BioGridDataDec2018.ipynb mapping_table.csv        result_mapped.txt


In [8]:
os.getcwd()

'/Users/ken/Projects/Dec2018Notebooks'

In [168]:
!cat result_mapped.txt | cut -f 1,2 | head

yourlist:M201812188471C63D39733769F8E060B506551E1201F3EBJ	Entry
131072	Q86XK7
131079	Q5VW00
131082	Q6ZR62
131091	Q5GFL6
131092	Q8WY41
131098	A6NHT5
131110	Q6ZRI0
131112	P59894
131113	Q8IXP5


In [5]:
idc.show_mapping_table()

Scraping the web..
+----------------------------+------------------------+-----------+
|            Name            |      Abbreviation      | Direction |
+----------------------------+------------------------+-----------+
|      UniProtKB AC/ID       |         ACC+ID         |    from   |
|        UniProtKB AC        |          ACC           |    both   |
|        UniProtKB ID        |           ID           |    both   |
|          UniParc           |         UPARC          |    both   |
|          UniRef50          |          NF50          |    both   |
|          UniRef90          |          NF90          |    both   |
|         UniRef100          |         NF100          |    both   |
|         Gene name          |        GENENAME        |    both   |
|     EMBL/GenBank/DDBJ      |        EMBL_ID         |    both   |
|   EMBL/GenBank/DDBJ CDS    |          EMBL          |    both   |
|    Entrez Gene (GeneID)    |     P_ENTREZGENEID     |    both   |
|         GI number          

In [6]:
# Datadir = '/Users/ken/Documents/From_KEN_MAC/2018Data'
Datadir = '/media/user/FAT_BOY/2018LITTLEMAN_EXT/2018Data' 

In [7]:
biogridfile = os.path.join(Datadir, "BIOGRID-ALL-3.5.165.tab2.txt")

In [16]:
df = pd.read_csv(biogridfile, sep='\t')

/home/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
df.head(2)

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,9006895,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,11309420,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID


## Get interactors

In [18]:
df.columns

Index(['#BioGRID Interaction ID', 'Entrez Gene Interactor A',
       'Entrez Gene Interactor B', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Official Symbol Interactor A',
       'Official Symbol Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Pubmed ID',
       'Organism Interactor A', 'Organism Interactor B', 'Throughput', 'Score',
       'Modification', 'Phenotypes', 'Qualifications', 'Tags',
       'Source Database'],
      dtype='object')

In [19]:
len(df['Organism Interactor A'].unique().tolist())

62

In [20]:
df.shape

(1583787, 24)

## Get Human Interactions

In [21]:
human_mask = (df['Organism Interactor A'] == 9606) & (df['Organism Interactor B'] == 9606)

In [22]:
df_human = df.loc[human_mask].reset_index()

In [26]:
df_human = df_human.drop(['index'], axis=1)

In [27]:
df_human.shape

(409173, 24)

In [28]:
df_human.head(2)

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,9006895,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,11309420,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID


In [29]:
df_human['Experimental System'].unique()

array(['Two-hybrid', 'Affinity Capture-Western', 'Reconstituted Complex',
       'FRET', 'Co-purification', 'Protein-peptide', 'Co-localization',
       'Affinity Capture-MS', 'Biochemical Activity',
       'Affinity Capture-RNA', 'Co-crystal Structure', 'Far Western',
       'Phenotypic Enhancement', 'Phenotypic Suppression',
       'Co-fractionation', 'Protein-RNA', 'Synthetic Rescue',
       'Affinity Capture-Luminescence', 'PCA', 'Dosage Rescue',
       'Negative Genetic', 'Dosage Lethality', 'Synthetic Growth Defect',
       'Proximity Label-MS', 'Synthetic Lethality', 'Positive Genetic'],
      dtype=object)

In [33]:
sub_mask = ['BioGRID ID Interactor A','BioGRID ID Interactor B', 
            'Experimental System Type', 'Score', 'Pubmed ID']

In [34]:
df_human_exp = df_human.loc[:,sub_mask]

In [40]:
df_human_exp.columns = ["BiogridA", "BiogridB", "ExperimentalSystem", "Score" ,"PubmedId"]

In [41]:
df_human_exp.head(2)

,BiogridA,BiogridB,ExperimentalSystem,Score,PubmedId
0,112315,108607,physical,-,9006895
1,124185,106603,physical,-,11309420


In [ ]:
# df_human_exp['Score'].unique().tolist()

In [42]:
lsa = df_human_exp['BiogridA'].unique().tolist()
lsb = df_human_exp['BiogridB'].unique().tolist()

In [43]:
lsfull = set(lsa).union(set(lsb))

In [44]:
len(lsfull)

17299

In [45]:
lsN = list(set(lsa + lsb))

In [46]:
len(lsN)

17299

## Write BioGRID ids to a file

In [49]:
with open("bioigrid_list.txt", "w") as infile:
    for i in list(lsfull):
        infile.write("{}\n".format(i))
        

## Read mapped BioGRID IDs

In [53]:
mappedfile = os.path.join(os.path.curdir, "biogrid_to_uniprot.tab.gz")

In [56]:
dfmap = pd.read_csv(mappedfile, compression='gzip', sep='\t')

In [57]:
dfmap.head(2)

,yourlist:M20190627E5A08BB0B2D1C45B0C7BC3B55FD265562430386,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,131072,Q86XK7,VSIG1_HUMAN,reviewed,V-set and immunoglobulin domain-containing pro...,VSIG1 GPA34,Homo sapiens (Human),387
1,131079,Q5VW00,DC122_HUMAN,reviewed,DDB1- and CUL4-associated factor 12-like prote...,DCAF12L2 WDR40C,Homo sapiens (Human),463


In [62]:
dfmap.rename(columns={'yourlist:M20190627E5A08BB0B2D1C45B0C7BC3B55FD265562430386':'BiogridID', 
                     'Entry':"UniprotID"}, inplace=True)

dfmap.head(2)

In [104]:
# Split  cells
dfmap.BiogridID.apply(pd.Series) \
.merge(dfmap, right_index = True, left_index = True) \
.drop(['BiogridID'], axis=1) \ 
.melt(id_vars = ['uniprot', 'id'], value_name = 'biogrid') \
.drop('variable', axis=1) \
.dropna()

SyntaxError: unexpected character after line continuation character (<ipython-input-104-9cf2078aa2e2>, line 2)

In [122]:
dfmap.head(2)

,BiogridID,UniprotID,Entry name,Status,Protein names,Gene names,Organism,Length
0,131072,Q86XK7,VSIG1_HUMAN,reviewed,V-set and immunoglobulin domain-containing pro...,VSIG1 GPA34,Homo sapiens (Human),387
1,131079,Q5VW00,DC122_HUMAN,reviewed,DDB1- and CUL4-associated factor 12-like prote...,DCAF12L2 WDR40C,Homo sapiens (Human),463


In [120]:
dfmap.BiogridID.apply(pd.Series).\
    merge(dfmap, right_index = True, left_index = True).\
    drop(['BiogridID'], axis=1).\
    melt(id_vars = ['UniprotID', 0], value_name = 'Gene').\
    drop('variable', axis=1)

,UniprotID,0,Gene
0,Q86XK7,131072,VSIG1_HUMAN
1,Q5VW00,131079,DC122_HUMAN
2,Q6ZR62,131082,RTL4_HUMAN
3,Q5GFL6,131091,VWA2_HUMAN
4,Q8WY41,131092,NANO1_HUMAN
5,A6NHT5,131098,HMX3_HUMAN
6,Q6ZRI0,131110,OTOG_HUMAN
7,M0R2J8,131112,DCDC1_HUMAN
8,Q8IXP5,131113,CK053_HUMAN
9,Q6MZM0,131121,HPHL1_HUMAN


In [124]:
dfmap.shape

(16146, 8)

In [97]:
dfmapR = pd.DataFrame(dfmap.UniprotID.str.split(','), index=dfmap.index).stack()

In [98]:
dfmapR

0      UniprotID    [Q86XK7]
1      UniprotID    [Q5VW00]
2      UniprotID    [Q6ZR62]
3      UniprotID    [Q5GFL6]
4      UniprotID    [Q8WY41]
5      UniprotID    [A6NHT5]
6      UniprotID    [Q6ZRI0]
7      UniprotID    [M0R2J8]
8      UniprotID    [Q8IXP5]
9      UniprotID    [Q6MZM0]
10     UniprotID    [Q6XYQ8]
11     UniprotID    [P0C7M7]
12     UniprotID    [Q75WM6]
13     UniprotID    [Q5SZK8]
14     UniprotID    [Q6P3R8]
15     UniprotID    [Q9NYA3]
16     UniprotID    [Q6NX45]
17     UniprotID    [Q68DA7]
18     UniprotID    [A6NNT2]
19     UniprotID    [Q63HK3]
20     UniprotID    [P0C7T5]
21     UniprotID    [Q7Z443]
22     UniprotID    [Q496F6]
23     UniprotID    [Q2TAL5]
24     UniprotID    [Q9H009]
25     UniprotID    [Q7Z3Y8]
26     UniprotID    [Q6IEE8]
27     UniprotID    [Q6ZMT1]
28     UniprotID    [Q6ZVX7]
29     UniprotID    [Q494X3]
                      ...   
16115  UniprotID    [Q6IQ16]
16116  UniprotID    [Q4G0S4]
16117  UniprotID    [A8MZ97]
16118  Uniprot

In [77]:
dfmap.loc[dfmap["BiogridID"] == '107463' ] 

,BiogridID,UniprotID,Entry name,Status,Protein names,Gene names,Organism,Length
1283,107463,P42771,CDN2A_HUMAN,reviewed,Cyclin-dependent kinase inhibitor 2A (Cyclin-d...,CDKN2A CDKN2 MTS1,Homo sapiens (Human),156
1284,107463,Q8N726,ARF_HUMAN,reviewed,Tumor suppressor ARF (Alternative reading fram...,CDKN2A CDKN2 MLM,Homo sapiens (Human),132


## Create a mapping dictionary object

In [66]:
idgroup = dfmap.groupby(['BiogridID'])['UniprotID']

In [69]:
dict_biogrid = {k:list(v) for k, v in idgroup}

In [70]:
dict_biogrid

{'106523': ['P04217'],
 '106524': ['P01023'],
 '106527': ['P18440'],
 '106528': ['P11245'],
 '106530': ['P01011'],
 '106532': ['Q13685'],
 '106533': ['Q16613'],
 '106534': ['P49588'],
 '106536': ['P80404'],
 '106537': ['O95477'],
 '106538': ['Q9BZC7'],
 '106539': ['Q99758'],
 '106540': ['O75027'],
 '106541': ['Q8NE71'],
 '106542': ['P78363'],
 '106543': ['P00519'],
 '106544': ['P19801'],
 '106545': ['P42684'],
 '106546': ['P16442'],
 '106547': ['Q12979'],
 '106548': ['P09110'],
 '106549': ['Q13085'],
 '106550': ['O00763'],
 '106551': ['P28330'],
 '106552': ['P11310'],
 '106553': ['P16219'],
 '106554': ['P45954'],
 '106555': ['P49748'],
 '106556': ['P24752'],
 '106557': ['Q9BWD1'],
 '106558': ['Q16515'],
 '106559': ['P78348'],
 '106561': ['P22303'],
 '106563': ['P53396'],
 '106564': ['P21399'],
 '106565': ['P10323'],
 '106566': ['Q99798'],
 '106567': ['Q15067'],
 '106568': ['P24666'],
 '106569': ['P11117'],
 '106570': ['P13686'],
 '106571': ['P15309'],
 '106573': ['P68133'],
 '106574': 

In [78]:
dfmap.loc[dfmap['UniprotID'] == 'P0DML3']

,BiogridID,UniprotID,Entry name,Status,Protein names,Gene names,Organism,Length
1537,"107829,107830",P0DML3,CSH2_HUMAN,reviewed,Chorionic somatomammotropin hormone 2 (Choriom...,CSH2,Homo sapiens (Human),217


In [179]:
lsboth = set(lsa).intersection(set(lsb))

In [180]:
lsBiogridProts = list(lsfull)[:]

In [181]:
len(lsboth)

12268

In [182]:
lsBiogridProts.__len__()

17299

## Break Long Lists into Chunks

In [183]:
def chunks(l, n=1000):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [192]:
lschunks = chunks(lsBiogridProts)
topchunks = (i[:10] for i in lschunks)

try:
    mapped = (idc.idmapping('BIOGRID_ID', 'ACC', lsids=j) for j in topchunks)
except KeyboardInterrupt as ke:
    print("Interrupted by the user")


In [193]:
next(mapped)

In [194]:
!ls

2018-12-18.21:00:00_result_mapped.txt mapping_table.csv
BioGridDataDec2018.ipynb              result_mapped.txt


## Process chunks

In [67]:
%%time
def process_chunks(idfro, idto, lstchunks):
    count = 0
    for chunk in lstchunks:
        print(len(chunk))
#         time.sleep(2)
        count +=1
#         yield len(chunk)
#         print("processing chunk_{}...".format(count))
# #         yield chunk[:10]
#         yield idc.idmapping(idfro,idto, lsids=chunk)
        

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.1 µs


In [68]:
%%time
my_chunks =  process_chunks('BIOGRID_ID', 'ACC', lsChunks)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11 µs


In [70]:
my_chunks

In [57]:
list(p)

[]

In [40]:
!ls

2018-12-18.17:21:23_result_mapped.txt mapping_table.csv
2018-12-18.21:00:00_result_mapped.txt result_mapped.txt
BioGridDataDec2018.ipynb


In [85]:
%%time
idc.idmapping('BIOGRID_ID','ACC', lsids=lsBiogridProts)

CPU times: user 27.5 ms, sys: 6.32 ms, total: 33.8 ms
Wall time: 4.05 s


In [73]:
!ls

BioGridDataDec2018.ipynb mapping_table.csv        result_mapped.txt


In [65]:
!cut -f 1,2 result_mapped.txt | head 

yourlist:M20181218E579DAD5D4BBAF98E82B06D27107170A049E1FK	Entry
131072	Q86XK7
131079	Q5VW00
131082	Q6ZR62
131091	Q5GFL6
131092	Q8WY41
131098	A6NHT5
131110	Q6ZRI0
131112	P59894
131113	Q8IXP5


In [109]:
" ".join(list(map(lambda x: str(x), IDLIST)))

'P13368 P20806 Q9UM73 P97793'

In [139]:
idc.idconversion.IDLIST

['P13368', 'P20806', 'Q9UM73', 'P97793']

In [217]:
la = (i for i in range(10))

In [211]:
next(la)

3

In [218]:
squares = (i**2 for i in la)

In [224]:
next(squares)

25